TO DO:
- xoak those lat/lon and get only eddies within the mask
- create the statistics
- do the time series
- do the histograms!

In [1]:
from glob import glob
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import xgcm
from xorca.lib import load_xorca_dataset
import pickle
import eddytools as et
from cmocean import cm
from dask.diagnostics import ProgressBar
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import timedelta
import matplotlib.colors as colors
import seawater as sw
import math
import scipy.stats as stats
from matplotlib import axes
import xoak

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [2]:
import dask, dask.distributed
import dask_jobqueue
cluster = dask_jobqueue.SLURMCluster(cores=8,memory='60GB',processes=1,queue='large96:shared', walltime='6:00:00',interface='ib0',local_directory='$TMPDIR')
client = dask.distributed.Client(cluster)
cluster.scale(jobs=8)

In [3]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.246.8.1:8787/status,
Dashboard: http://10.246.8.1:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.246.8.1:39207,Workers: 0
Dashboard: http://10.246.8.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
def create_stat(model, year_start, year_finished = 2018): # UPDATED 09.09.2022 - > ONLY TYPED0
    
    """ 
        model == 0 > EXP05 elif == 1 EXP06
        typed == 0 > ACC / elif == 1 WHOLE REGION
    """
    
    folder = 'EXP05' if model == 0 else 'EXP06'
    
    with open(datapath + folder + '/detected/track_dict_' + str(year_start) + '0101_' + str(year_finished) + '1231_tracks_typed0.pickle', 'rb') as f:
        tracks = pickle.load(f)
    f.close()     
    
    eddy_stat = pd.DataFrame.from_dict(tracks, orient = 'index').drop(['eddy_j', 'eddy_i'], axis=1)
    
    for i in eddy_stat.index:
        eddy_stat.loc[i, 'life_time'] = pd.to_datetime(eddy_stat.time.loc[i][-1]) - pd.to_datetime(eddy_stat.time.loc[i][0])
    
    eddy_stat = eddy_stat.loc[(eddy_stat['exist_at_start'] ==  False) & (eddy_stat.terminated == True)]
    
    for i in eddy_stat.index:
        lat = [eddy_stat.loc[i, 'lat'][0], eddy_stat.loc[i, 'lat'][-1]]
        lon = [eddy_stat.loc[i, 'lon'][0], eddy_stat.loc[i, 'lon'][-1]]
        eddy_stat.loc[i, 'distance'] = sw.extras.dist(lat,lon)[0]
        #eddy_stat.loc[i, 'speed'] = eddy_stat.loc[i, 'distance'] / eddy_stat.loc[i, 'life_time']
        eddy_stat.loc[i, 'speed'] = (eddy_stat.loc[i, 'distance'] *1e5)/ (eddy_stat.loc[i, 'life_time'].days * 24 * 3600) # cm/s
#         eddy_stat.loc[i, 'angle'] = (sw.extras.dist(eddy_stat.loc[i, 'lat'], eddy_stat.loc[i, 'lon'])[1]).mean()
        eddy_stat.loc[i, 'area_mean'] = eddy_stat.loc[i, 'area'].mean()
        eddy_stat.loc[i, 'area_max'] = eddy_stat.loc[i, 'area'].max()
        eddy_stat.loc[i, 'amp_mean'] = eddy_stat.loc[i, 'amp'].mean()
        eddy_stat.loc[i, 'amp_max'] = eddy_stat.loc[i, 'amp'].max()
        eddy_stat.loc[i, 'scale_mean'] = eddy_stat.loc[i, 'scale'].mean()
        eddy_stat.loc[i, 'scale_min'] = eddy_stat.loc[i, 'scale'].min()
        eddy_stat.loc[i, 'scale_max'] = eddy_stat.loc[i, 'scale'].max()
        eddy_stat.loc[i, 'year_i'] = pd.to_datetime(eddy_stat.time.loc[i][0]).year
        eddy_stat.loc[i, 'year_f'] = pd.to_datetime(eddy_stat.time.loc[i][-1]).year
    
    eddy_stat = eddy_stat.drop(labels=['lon','lat','amp','area','scale','exist_at_start','terminated'],axis=1)

    eddy_stat = eddy_stat.loc[(eddy_stat['life_time'].astype('timedelta64[D]') >= 30)]
    eddy_stat.loc[:, 'life_time'] = eddy_stat.loc[:, 'life_time'].astype('timedelta64[D]')
        
    eddy_stat.loc[:,'scale_mean'] *= 2
    eddy_stat.loc[:,'scale_max'] *= 2

    eddy_stat = eddy_stat.drop('time', axis=1)

    
    eddy_stat.to_pickle(path= datapath + folder + '/detected/stat_' + folder + '_acc_'+ str(year_start) + '_' + str(year_finished) +'.pkl')

    return eddy_stat

In [ ]:
create_stat(model=1, year_start=1993, year_finished=2018)

### path to data

In [4]:
datapath = '/scratch/usr/shkifmmp/manusc/data/'

path_EXP05_old = '/scratch/usr/shkifmmp/master/data/ORION10/eddies_detect/acc/'

mask_acc = xr.open_dataset('/scratch/usr/shkifmmp/master/data/ORION10/mask/mask_nest_int_2d.nc', chunks={"y":100, "x":100})

In [5]:
mask_plot = mask_acc.mask_nest.where(mask_acc.mask_nest ==0, other=1)

mask_plot = mask_plot.where(mask_plot.nav_lat >= -65, other=0)

In [70]:
mask_plot.nav_lon.values

array([[ 75.5       ,  75.59999847,  75.69999695, ..., 431.40000153,
        431.5       , 431.59999847],
       [ 75.5       ,  75.59999847,  75.69999695, ..., 431.40000153,
        431.5       , 431.59999847],
       [ 75.5       ,  75.59999847,  75.69999695, ..., 431.40000153,
        431.5       , 431.59999847],
       ...,
       [ 75.5       ,  75.59999847,  75.69999695, ..., 431.40000153,
        431.5       , 431.59999847],
       [ 75.5       ,  75.59999847,  75.69999695, ..., 431.40000153,
        431.5       , 431.59999847],
       [ 75.5       ,  75.59999847,  75.69999695, ..., 431.40000153,
        431.5       , 431.59999847]])

In [71]:
431-75

356

In [5]:
def create_stat(model, typed, year_finished = 2018):
    
    """ 
        model == 0 > EXP05 elif == 1 EXP06
        typed == 0 > ACC / elif == 1 WHOLE REGION
    """
    
    folder = 'EXP05' if model == 0 else 'EXP06'
    
    if (typed == 0) & (model == 0):
        with open(path_EXP05_old + 'track_dict_19580101_' + str(year_finished) + '1231_tracks_typed' + str(typed) 
                  + '.pickle', 'rb') as f:
            tracks = pickle.load(f)
        f.close()
    else:
        with open(datapath + folder + '/detected/track_dict_19580101_' + str(year_finished) + '1231_tracks_typed' + str(typed) 
                  + '.pickle', 'rb') as f:
            tracks = pickle.load(f)
        f.close()     
    
    eddy_stat = pd.DataFrame.from_dict(tracks, orient = 'index').drop(['eddy_j', 'eddy_i'], axis=1)
    
    for i in eddy_stat.index:
        eddy_stat.iloc[i, 'life_time'] = pd.to_datetime(eddy_stat.time.iloc[i][-1]) - pd.to_datetime(eddy_stat.time.iloc[i][0])
    
    eddy_stat = eddy_stat.loc[(eddy_stat['exist_at_start'] ==  False) & (eddy_stat.terminated == True)]
    
    for i in eddy_stat.index:
        lat = [eddy_stat.loc[i, 'lat'][0], eddy_stat.loc[i, 'lat'][-1]]
        lon = [eddy_stat.loc[i, 'lon'][0], eddy_stat.loc[i, 'lon'][-1]]
        eddy_stat.loc[i, 'distance'] = sw.extras.dist(lat,lon)[0]
        #eddy_stat.loc[i, 'speed'] = eddy_stat.loc[i, 'distance'] / eddy_stat.loc[i, 'life_time']
        eddy_stat.loc[i, 'speed'] = (eddy_stat.loc[i, 'distance'] *1e5)/ (eddy_stat.loc[i, 'life_time'].days * 24 * 3600) # cm/s
#         eddy_stat.loc[i, 'angle'] = (sw.extras.dist(eddy_stat.loc[i, 'lat'], eddy_stat.loc[i, 'lon'])[1]).mean()
        eddy_stat.loc[i, 'area_mean'] = eddy_stat.loc[i, 'area'].mean()
        eddy_stat.loc[i, 'area_max'] = eddy_stat.loc[i, 'area'].max()
        eddy_stat.loc[i, 'amp_mean'] = eddy_stat.loc[i, 'amp'].mean()
        eddy_stat.loc[i, 'amp_max'] = eddy_stat.loc[i, 'amp'].max()
        eddy_stat.loc[i, 'scale_mean'] = eddy_stat.loc[i, 'scale'].mean()
        eddy_stat.loc[i, 'scale_min'] = eddy_stat.loc[i, 'scale'].min()
        eddy_stat.loc[i, 'scale_max'] = eddy_stat.loc[i, 'scale'].max()
        eddy_stat.loc[i, 'year_i'] = pd.to_datetime(eddy_stat.time.loc[i][0]).year
        eddy_stat.loc[i, 'year_f'] = pd.to_datetime(eddy_stat.time.loc[i][-1]).year
        
    eddy_stat.to_pickle(path= datapath + folder + '/detected/stat_' + folder + '_acc_1958_' + str(year_finished) +'.pkl')
    
    return eddy_stat

In [14]:
folder='EXP06'
year_finished=1996

datapath + folder + '/detected/stat_' + folder + '_acc_1958_' + str(year_finished) +'.pkl'

'/scratch/usr/shkifmmp/manusc/data/EXP06/detected/stat_EXP06_acc_1958_1996.pkl'

In [4]:
def create_stat_old():
    with open(path_EXP05_old + 'case9_19780101_19971231_tracks_OW0.3_77.5_69.5.pickle', 'rb') as f:
            tracks = pickle.load(f)
    f.close()
    
    eddy_stat = pd.DataFrame.from_dict(tracks, orient = 'index').drop(['eddy_j', 'eddy_i'], axis=1)
    
    for i in eddy_stat.index:
        eddy_stat.loc[i, 'life_time'] = pd.to_datetime(eddy_stat.time.iloc[i][-1]) - pd.to_datetime(eddy_stat.time.iloc[i][0])
    
    eddy_stat = eddy_stat.loc[(eddy_stat['exist_at_start'] ==  False) & (eddy_stat.terminated == True)]
    
    for i in eddy_stat.index:
        lat = [eddy_stat.loc[i, 'lat'][0], eddy_stat.loc[i, 'lat'][-1]]
        lon = [eddy_stat.loc[i, 'lon'][0], eddy_stat.loc[i, 'lon'][-1]]
        eddy_stat.loc[i, 'distance'] = sw.extras.dist(lat,lon)[0]
        #eddy_stat.loc[i, 'speed'] = eddy_stat.loc[i, 'distance'] / eddy_stat.loc[i, 'life_time']
        eddy_stat.loc[i, 'speed'] = (eddy_stat.loc[i, 'distance'] *1e5)/ (eddy_stat.loc[i, 'life_time'].days * 24 * 3600) # cm/s
#         eddy_stat.loc[i, 'angle'] = (sw.extras.dist(eddy_stat.loc[i, 'lat'], eddy_stat.loc[i, 'lon'])[1]).mean()
        eddy_stat.loc[i, 'area_mean'] = eddy_stat.loc[i, 'area'].mean()
        eddy_stat.loc[i, 'area_max'] = eddy_stat.loc[i, 'area'].max()
        eddy_stat.loc[i, 'amp_mean'] = eddy_stat.loc[i, 'amp'].mean()
        eddy_stat.loc[i, 'amp_max'] = eddy_stat.loc[i, 'amp'].max()
        eddy_stat.loc[i, 'area_max'] = eddy_stat.loc[i, 'area'].max()
        eddy_stat.loc[i, 'scale_mean'] = eddy_stat.loc[i, 'scale'].mean()
        eddy_stat.loc[i, 'scale_min'] = eddy_stat.loc[i, 'scale'].min()
        eddy_stat.loc[i, 'scale_max'] = eddy_stat.loc[i, 'scale'].max()
        
    return eddy_stat

In [ ]:
stat_06 = create_stat(model=1, typed=0, year_finished=2018)

In [ ]:
stat_05_78.to_pickle(path= datapath + 'EXP05/detected/stat_05_acc_1978_1997.pkl')

In [ ]:
for i in stat_05_78.index:
        stat_05_78.loc[i, 'year_i'] = pd.to_datetime(stat_05_78.time.iloc[i][0]).year
        stat_05_78.loc[i, 'year_f'] = pd.to_datetime(stat_05_78.time.iloc[i][-1]).year

In [ ]:
folder
stat_05_78.to_pickle(path= datapath + 'EXP05/detected/stat_05_acc_1978_1997.pkl')

### time series

### OBSERVATIONS

In [28]:
# def adjust_longitude(long): # change to 0-360 longitude range FOR THE META DATA
#     func = lambda x: (-x + 180) % 360
#     return xr.apply_ufunc(func, long)

In [24]:
# meta20_dt = xr.open_dataset(datapath + 'obs/eddy_trajectory_dt_2.0_19930101_20200307.nc')

# track_eddy = xr.open_dataset(datapath + 'obs/trackeddy_V1-0_global_eddies.nc').drop(['area_gaussian','angle','angle_gaussian',
#                                                                                     'gaussian_spread_x','gaussian_spread_y', 'level'])

# t_eddy = track_eddy.where(track_eddy.maxima_loc_y<= -30, drop=True)

# t_eddy.to_netcdf(path= datapath + 'obs/trackeddy_30S.nc')

# meta = meta20_dt.where(meta20_dt.latitude<= -30, drop=True)

# meta.to_netcdf(path= datapath + 'obs/eddy_DT_30S.nc')

In [6]:
meta = xr.open_dataset(datapath + 'obs/eddy_DT_30S.nc')

In [ ]:
# x = np.arange(-47,372,1) # change to 0-360 longitude range
# y = (-x + 180) % 360
# plt.plot(x,y)

In [ ]:
# meta.update({'longitude': adjust_longitude(meta.longitude)})

In [47]:
# meta.to_netcdf(path= datapath + 'obs/eddy_DT_30S.nc')

In [9]:
meta.longitude.max()

<xarray.DataArray 'longitude' ()>
array(359.99966431)

In [7]:
track_e = xr.open_dataset(datapath + 'obs/trackeddy_30S.nc')

In [7]:
mask_test = mask_plot.where(mask_plot.nav_lon <=360, drop=True)

mask_test_aux = mask_plot.where(mask_plot.nav_lon >360, drop=True)

mask_test_aux.coords['nav_lon'].values = mask_test_aux.nav_lon.values - 360

mask_obs= xr.concat([mask_test_aux, mask_test], 'x')

In [ ]:
# mask_obs.nav_lon.plot()

In [8]:
mask_obs.xoak.set_index(['nav_lon','nav_lat'], "s2point")

In [11]:
lati = meta.latitude.compute()

In [12]:
long = meta.longitude.compute()

In [13]:
long

<xarray.DataArray 'longitude' (obs: 13358281)>
array([160.6825  , 160.7583  , 160.8202  , ...,  12.897995,  12.625   ,
        12.475998], dtype=float32)
Dimensions without coordinates: obs

In [14]:
meta_xo = mask_obs.xoak.sel(nav_lat= lati, nav_lon=long).values

KilledWorker: ("('lambda-a330da366895501f3e676c7724466c6d', 0)", <WorkerState 'tcp://10.246.8.1:43445', name: SLURMCluster-0, memory: 0, processing: 1>)

In [ ]:
meta_xo = xr.DataArray(data=meta_xo, dims=['obs'])

meta = meta.where(meta_xo, drop=True)

meta.to_netcdf(path=datapath + 'obs/meta_xoak.nc')

In [12]:
datapath

'/scratch/usr/shkifmmp/manusc/data/'

In [ ]:
track_e_xo = mask_obs.xoak.sel(nav_lat= track_e.latitude, nav_lon=track_e.longitude).values

In [ ]:
# BEFORE RUNNING: CHANNGE THE 'OBS' TO THE EQUIVALENT IN TRACK_E !!!
track_e_xo = xr.DataArray(data=track_e_xo, dims=['obs'])

track_e = meta.where(track_e_xo, drop=True)

track_e.to_netcdf(path=path_obs + 'track_e_xoak.nc')

In [47]:
(pd.to_datetime(meta.where(meta.track == 0, drop = True).time[-1].values) - pd.to_datetime(meta.where(meta.track == 0, drop = True).time[0].values))

Timedelta('73 days 00:00:00')

In [ ]:
(pd.to_datetime(meta20_dt.where(meta20_dt.track == 1,drop=True).groupby('track').first().time) - pd.to_datetime(meta20_dt.where(meta20_dt.track == 1,drop=True).groupby('track').last().time)).astype().astype

In [2]:
meta

NameError: name 'meta' is not defined

In [29]:
track_e = track_e.rename({'maxima_loc_x': 'longitude',
               'maxima_loc_y': 'latitude',
               'eddy_amplitude': 'amplitude',
               'area_eddy': 'area'})

In [31]:
meta.rename({})

<xarray.Dataset>
Dimensions:             (obs: 13358281)
Dimensions without coordinates: obs
Data variables:
    amplitude           (obs) float32 ...
    cyclonic_type       (obs) float64 ...
    latitude            (obs) float32 ...
    longitude           (obs) float32 ...
    observation_flag    (obs) float64 ...
    observation_number  (obs) float64 ...
    speed_average       (obs) float32 ...
    speed_radius        (obs) float32 ...
    time                (obs) datetime64[ns] ...
    track               (obs) float64 ...
Attributes: (12/17)
    title:                     Mesoscale Eddy Trajectories Atlas in Altimeter...
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    comment:                   Surface product; mesoscale eddies
    framework_used:            https://github.com/AntSimi/py-eddy-tracker
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    date_created:              2020-12-15T16:25:59Z
    ...                        ...
    creator_url:               http://www.aviso.altimetry.fr
    creator_email:             aviso@altimetry.fr
    license:                   http://www.aviso.altimetry.fr/fileadmin/docume...
    summary:                   This dataset contains mesoscale eddy trajector...
    input_product_reference:   https://www.aviso.altimetry.fr/en/data/product...
    product_version:           2.0

In [21]:
track_e.isel(time=200).time_tracking.values.astype('timedelta64[D]')

numpy.timedelta64(1,'D')

In [28]:
track_e.isel(time=20).neddy

<xarray.DataArray 'neddy' ()>
array(2625642.)
Coordinates:
    time     datetime64[ns] 1994-09-15
Attributes:
    description:  observation sequence number, individual eddy identifier
    long_name:    individual eddy identifier
    max:          337204323
    min:          0
    units:        1

In [11]:
meta

<xarray.Dataset>
Dimensions:             (obs: 13358281)
Dimensions without coordinates: obs
Data variables:
    amplitude           (obs) float32 ...
    cyclonic_type       (obs) float64 ...
    latitude            (obs) float32 ...
    longitude           (obs) float32 ...
    observation_flag    (obs) float64 ...
    observation_number  (obs) float64 ...
    speed_average       (obs) float32 ...
    speed_radius        (obs) float32 ...
    time                (obs) datetime64[ns] ...
    track               (obs) float64 ...
Attributes: (12/17)
    title:                     Mesoscale Eddy Trajectories Atlas in Altimeter...
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    comment:                   Surface product; mesoscale eddies
    framework_used:            https://github.com/AntSimi/py-eddy-tracker
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    date_created:              2020-12-15T16:25:59Z
    ...                        ...
    creator_url:               http://www.aviso.altimetry.fr
    creator_email:             aviso@altimetry.fr
    license:                   http://www.aviso.altimetry.fr/fileadmin/docume...
    summary:                   This dataset contains mesoscale eddy trajector...
    input_product_reference:   https://www.aviso.altimetry.fr/en/data/product...
    product_version:           2.0

In [66]:
# first get eddies within area
def obs_proc(data):
    first = data.groupby('track').first()
    last = data.groupby('track').last()
    mean = data.groupby('track').mean()
    maxi = data.groupby('track').max()
    
    life_time = pd.to_datetime(last.time) - pd.to_datetime(last.time)
    amp_max, amp_mean = 
    speed = mean.speed_average
    lat, lon = first
    distance = sw.extras.dist([lat, lon])
    

SyntaxError: invalid syntax (1494480485.py, line 9)

In [34]:
eddy_stat.loc[i, 'distance'] = sw.extras.dist(lat,lon)[0]

Signature: sw.extras.dist(lat, lon, units='km')
Docstring:
Calculate distance between two positions on globe using the "Plane
Sailing" method. Also uses simple geometry to calculate the bearing of
the path between position pairs.

Parameters
----------
lat : array_like
      decimal degrees (+ve N, -ve S) [- 90.. +90]
lon : array_like
      decimal degrees (+ve E, -ve W) [-180..+180]
units : string, optional
        default kilometers

Returns
-------
dist : array_like
       distance between positions in units
phaseangle : array_like
             angle of line between stations with x axis (East).
             Range of values are -180..+180. (E=0, N=90, S=-90)

Examples
--------
>>> import seawater as sw
>>> sw.dist(0, [-179, 180])
(array([ 111.12]), array([ 180.]))
>>> lon = [35, 35]
>>> lat = [41, 40]
>>> sw.dist(lat, lon)
(array([ 111.12]), array([-90.]))
>>> # Create a distance vector.
>>> lon = np.arange(30,40,1)
>>> lat = 35
>>> np.cumsum(np.append(0, sw.dist(lat, lon, units='km'

array([[-67.9824  , -67.9824  , -67.9824  , ..., -67.9824  , -67.9824  ,
        -67.9824  ],
       [-67.94488 , -67.94488 , -67.94488 , ..., -67.94488 , -67.94488 ,
        -67.94488 ],
       [-67.907295, -67.907295, -67.907295, ..., -67.907295, -67.907295,
        -67.907295],
       ...,
       [-29.936827, -29.936827, -29.936827, ..., -29.936827, -29.936827,
        -29.936827],
       [-29.85013 , -29.85013 , -29.85013 , ..., -29.85013 , -29.85013 ,
        -29.85013 ],
       [-29.763361, -29.763361, -29.763361, ..., -29.763361, -29.763361,
        -29.763361]], dtype=float32)